In [6]:
import cbpro
import pandas as pd
import time
import datetime
import random

In [7]:
client = cbpro.PublicClient()

In [8]:
client.get_product_ticker("ETH-USDT")

{'ask': '1495.84',
 'bid': '1495.75',
 'volume': '15821.52824143',
 'trade_id': 13103587,
 'price': '1496.01',
 'size': '0.2',
 'time': '2023-02-14T01:30:31.732553Z'}

In [9]:
def getData(start, end, coinType="ETH-USDT"):
    """
    The function helps with acquiring the historical data from the Coinbase Pro using python API.
    The cbpro API does not allow >300 requests at once. The API is likely to fail if it pulls too much data from the public client. 
    I have implement the function such that it sleeps 3-5 seconds before it request more data.
    """

    period = start + datetime.timedelta(seconds=3600*300)

    ls = []
    while start < end:
        print(start)
        # get historic rates of etheureum to USD tether at utc+0
        data = client.get_product_historic_rates(coinType, granularity=3600,start=start, end=period)
        # convert into a pandas dataframe
        df = pd.DataFrame(data, columns=['time', 'open', 'high', 'low', 'close', 'volume'])
        # convert the time column into a datetime object
        df['time'] = pd.to_datetime(df['time'], unit='s')
        # set the time column as the index
        df.set_index('time', inplace=True)

        ls.append(df)
        start = period
        period = period + datetime.timedelta(seconds=3600*300)

        time.sleep(random.uniform(3,5))
    temp = pd.concat(ls)
    temp.sort_values("time", ascending=True, inplace=True)
    # print(temp)
    outputFileName = str(coinType) + ".csv"
    temp.to_csv(outputFileName)

In [10]:
import datetime
start = datetime.datetime(2022, 11, 1, 0, 0, 0)
end = datetime.datetime(2023, 2, 1, 0, 0, 0)
getData(start, end, "ETH-USDT")

2022-11-01 00:00:00
2022-11-13 12:00:00
2022-11-26 00:00:00
2022-12-08 12:00:00
2022-12-21 00:00:00
2023-01-02 12:00:00
2023-01-15 00:00:00
2023-01-27 12:00:00
